# Training an XOR network using PySyft

In this notebook we'll be training an XOR network using PySyft. 
What's special about this is that the training  is DP- Protected


In [17]:
%load_ext autoreload
%autoreload 2
import syft as sy
import numpy as np
sy.logger.remove()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Logging into the domain Nodes

In [18]:
# We will login into Canada and Italy domain node
fb = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)
# fb = sy.login(email="info@openmined.org", password="changethis", port=8081)

Connecting to localhost... done! 	 Logging into facebook... done!


In [19]:
fb.datasets

Idx,Name,Description,Assets,Id
[0],Our training data for XOR networks!,Collected on Jan 27 2022,"[""training_data""] -> int64[""training_targets""] -> int64",9e999738-470a-4d1d-bcf8-c6292616a1f3
[1],Our training data for XOR networks!,Collected on Jan 27 2022,"[""training_data""] -> int64[""training_targets""] -> int64",3b1d6426-c093-4aba-8ba2-2f27f783079f
[2],Our training data for XOR networks!,Collected on Jan 27 2022,"[""training_data""] -> int64[""training_targets""] -> int64",4f031ab9-582c-452e-8f6e-974092073cb8
[3],Our training data for XOR networks!,Collected on Jan 27 2022,"[""training_data""] -> int64[""training_targets""] -> int64",5ed05695-30d1-4f63-8a70-ddc54b8b91b4


In [20]:
fb.privacy_budget

6019131.859025642

In [27]:
fb.privacy_budget

6017666.800938426

In [ ]:
from syft.core.node.common.node_service.user_manager.user_messages import (
    UpdateUserMessage,
)

# Upgrade admins budget
content = {"user_id": 1, "budget": 9_999_999}
fb._perform_grid_request(grid_msg=UpdateUserMessage, content=content)

fb.privacy_budget

In [21]:
fb_train_data = fb.datasets[-1]["training_data"]
fb_targets_data = fb.datasets[-1]["training_targets"]
X = fb_train_data
y = fb_targets_data

In [22]:
def relu(x,deriv=False):
    if deriv==True:
        return x>0
    return x*(x>0)

In [23]:
layer0_weights = 2*np.random.random((3,4)) - 1
layer1_weights = 2*np.random.random((4,1)) - 1

In [24]:
for j in range(2):
    # Forward propagation
    layer1_inputs = relu(X @ layer0_weights)  ; layer1_inputs.block
    layer2_inputs = relu(layer1_inputs @ layer1_weights) ; layer2_inputs.block 
    
    # Calculate errors
    layer2_inputs_delta = (y - layer2_inputs)* relu(layer2_inputs,deriv=True) ; layer2_inputs_delta.block
    layer1_inputs_delta = (layer2_inputs_delta@(layer1_weights.T)) * relu(layer1_inputs,deriv=True) ; layer1_inputs_delta.block
    
    # Update weights
    layer1_weights  = layer1_inputs.T @ layer2_inputs_delta + layer1_weights   ; layer1_weights.block
    layer0_weights =  X.T @ layer1_inputs_delta  + layer0_weights  ; layer0_weights.block

In [28]:
layer0_weights_dp = layer0_weights.publish(sigma=2e5)
layer1_weights_dp = layer1_weights.publish(sigma=2e5)

In [29]:
print(layer0_weights_dp)
print(layer1_weights_dp)

[[[-1.16310576 -6.4574685  -3.54603199 -2.39692892]
  [-5.64037664  0.05156036  3.38774476 -2.59111337]
  [ 1.62750189  1.88452134  0.09339615  0.67140623]]]
[[[  1.85296714]
  [  0.9981544 ]
  [ -4.2721291 ]
  [-10.01024287]]]


<hr>

<hr>
<hr>

Let's see how our privacy budget changed as a result of training for a single epoch:

In [ ]:
ds_domain1.privacy_budget

In [ ]:
ds_domain2.privacy_budget

And voila! We've trained a neural network using PySyft's adversaril differential privacy system and its secure multiparty computation system working in tandem.


<hr>
<hr>
Demo ends above- the cells below are for temporary Debugging:

In [ ]:
X

In [ ]:
layer0_weights

In [ ]:
layer1_inputs = relu(X @ layer0_weights)  ; layer1_inputs.block

In [ ]:
layer2_inputs = relu(layer1_inputs @ layer1_weights) ; layer2_inputs.block 

In [ ]:
layer2_inputs_delta = (y - layer2_inputs)* relu(layer2_inputs,deriv=True) ; layer2_inputs_delta.block

In [ ]:
layer1_inputs_delta = (layer2_inputs_delta@(layer1_weights.T)) * relu(layer1_inputs,deriv=True) ; layer1_inputs_delta.block

In [ ]:
layer1_weights  = layer1_inputs.T @ layer2_inputs_delta  + layer1_weights; layer1_weights.block

In [ ]:
layer0_weights =  X.T @ layer1_inputs_delta  + layer0_weights; layer0_weights.block

In [ ]:
res = X.T @ layer1_inputs_delta

In [ ]:
v1 = X.T
v2 = layer1_inputs_delta

In [ ]:
def get_val(val):
    t1 = val.child[0].get_copy()
    t2 = val.child[0].get_copy()
    return t1+t2

In [ ]:
v1 = get_val(v1)
v2 = get_val(v2)

In [ ]:
v1.child.min_vals.to_numpy() @ v2.child.min_vals.to_numpy()

In [ ]:
v2.child.min_vals

In [ ]:
layer0_weights_dp = layer0_weights.publish(sigma=1e4)
layer1_weights_dp = layer1_weights.publish(sigma=1e4)